In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [15]:
class MyTitanicModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = Dense(64, activation='relu')
        self.batch_normalization = BatchNormalization()
        self.dropout = Dropout(0.3)
        self.d2 = Dense(32, activation='relu')
        self.d3 = Dense(16, activation='relu')
        self.d4 = Dense(1, activation='sigmoid')

    def call(self, x):
        x = self.d1(x)
        x = self.batch_normalization(x)
        x = self.dropout(x)
        x = self.d2(x)
        x = self.dropout(x)
        x = self.d3(x)
        x = self.d4(x)
        return x

model = MyTitanicModel()

In [16]:
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.BinaryAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.BinaryAccuracy()

In [17]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_object(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_accuracy.update_state(y, predictions)


@tf.function
def test_step(x, y):
    predictions = model(x, training=False)
    loss = loss_object(y, predictions)

    test_loss.update_state(loss)
    test_accuracy.update_state(y, predictions)


## Load Data and train:

In [ ]:
data = pd.read_csv("./dataset/titanic_train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data["Sex"].replace(["female","male"],[0,1],inplace=True)
data.fillna(29.70,inplace=True)

In [20]:
from sklearn.model_selection import train_test_split

X = data[["Pclass","Sex","Age"]].values
Y = data[["Survived"]].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)

In [22]:
EPOCHS = 120

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for x_batch, y_batch in train_dataset:
        train_step(x_batch, y_batch)

    for x_batch, y_batch in test_dataset:
        test_step(x_batch, y_batch)

    print(
        f"Epoch {epoch + 1}, "
        f"Loss: {train_loss.result()}, "
        f"Accuracy: {train_accuracy.result()}, "
        f"Test Loss: {test_loss.result()}, "
        f"Test Accuracy: {test_accuracy.result()}"
    )


Epoch 1, Loss: 0.7212841510772705, Accuracy: 0.5667915344238281, Test Loss: 0.66058748960495, Test Accuracy: 0.644444465637207
Epoch 2, Loss: 0.6656659245491028, Accuracy: 0.6304619312286377, Test Loss: 0.6494237780570984, Test Accuracy: 0.6555555462837219
Epoch 3, Loss: 0.6482118964195251, Accuracy: 0.6304619312286377, Test Loss: 0.6364490389823914, Test Accuracy: 0.6777777671813965
Epoch 4, Loss: 0.6340879201889038, Accuracy: 0.6317103505134583, Test Loss: 0.6209044456481934, Test Accuracy: 0.7444444298744202
Epoch 5, Loss: 0.6208174228668213, Accuracy: 0.641697883605957, Test Loss: 0.6037247180938721, Test Accuracy: 0.7666666507720947
Epoch 6, Loss: 0.6080263257026672, Accuracy: 0.646691620349884, Test Loss: 0.587859570980072, Test Accuracy: 0.7777777910232544
Epoch 7, Loss: 0.5955365300178528, Accuracy: 0.6554307341575623, Test Loss: 0.5736343264579773, Test Accuracy: 0.800000011920929
Epoch 8, Loss: 0.5827145576477051, Accuracy: 0.6666666865348816, Test Loss: 0.5602245330810547, T